# 实现迭代协议

## 示例（实现一个迭代器以深度优先的模式遍历树的节点）

In [1]:
class Node:
    def __init__(self, value):
        self._vlaue = value
        self._children = []
    
    def __repr__(self):
        return 'Node({!r})'.format(self._vlaue)
    
    def add_child(self, node):
        self._children.append(node)
        
    def __iter__(self):
        return iter(self._children)
    
    def depth_first(self):
        yield self
        for c in self:
            yield from c.depth_first()

In [2]:
root = Node(0)
child1 = Node(1)
child2 = Node(2)
root.add_child(child1)
root.add_child(child2)
child1.add_child(Node(3))
child1.add_child(Node(4))
child2.add_child(Node(5))

for ch in root.depth_first():
    print(ch)

Node(0)
Node(1)
Node(3)
Node(4)
Node(2)
Node(5)


以上代码非常利于阅读，他首先产生出自身，然后迭代每个子节点，利用子节点的deep_first()方法（通过yield from）语句产生出其他元素。

- Python的迭代协议要求__iter__()返回一个特殊的迭代器对象，该对象必须实现__next__()方法，并使用StopIteration异常来通知迭代的完成。但是实现这样的对象会比较繁琐。例如下面的代码展示了另外一种depth_first()的实现，这里使用了一个相关联的迭代器类。

In [3]:
class Node:
    def __init__(self, value):
        self._value = value
        self._children = []
    
    def __repr__(self):
        return 'Node({!r})'.format(self._value)
    
    def add_child(self, node):
        self._children.append(node)
    
    def __iter__(self):
        return iter(self._children)
    
    def depth_first(self):
        return DepthFirstIterator(self)

    
class DepthFirstIterator(object):
    def __init__(self, start_node):
        self._node = start_node
        self._children_iter = None
        self._child_iter = None
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self._children_iter is None:
            self._children_iter = iter(self._node)
            return self._node
        elif self._child_iter:
            try:
                nextchild = next(self._child_iter)
                return nextchild
            except StopIteration:
                self._child_iter = None
                return next(self)
        else:
            self._child_iter = next(self._children_iter).depth_first()
            return next(self)
        

In [4]:
root = Node(0)
child1 = Node(1)
child2 = Node(2)
root.add_child(child1)
root.add_child(child2)
child1.add_child(Node(3))
child1.add_child(Node(4))
child2.add_child(Node(5))

for ch in root.depth_first():
    print(ch)

Node(0)
Node(1)
Node(3)
Node(4)
Node(2)
Node(5)


对比来看，DepthFirstIterator类的工作方式和生成器版本的实现相同但是却复杂了许多，把迭代器以生成器形式来定义就皆大欢喜了。